In [32]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [15]:
%fetchfile config.txt --print

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinpir       13

In [ ]:
# ESP8266 version

In [35]:
%sendtofile config.txt

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinscl       5
pinsda       4
pinpir       13

Sent 7 lines (238 bytes) to config.txt.


In [32]:
# ESP32 version
%sendtofile config.txt

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinscl       17
pinsda       16
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

Sent 6 lines (224 bytes) to config.txt.


In [33]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py


Sent 81 lines (2677 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [34]:

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# The redo for different connections isn't working so put the one 
# that is active into connection0

Sent 647 lines (23171 bytes) to mqtt_as.py.


In [28]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [34]:
#%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)
print("topicstem", topicstem)

def crc8(b):
    crc = 0xFF
    for c in b:
        crc ^= c
        for i in range(8):
            crc = 0xFF & (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1))
    return crc

if sys.platform == "esp32":
    i2c = machine.I2C(1, scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))
else:
    i2c = machine.I2C(scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))

topicscd41 = topicstem+"/scd41"
topicscd30 = topicstem+"/scd30"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"

async def SCD30task(client):
    try:
        i2c.writeto(0x61, bytes([0x02, 0x04]))   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, bytes([0x02, 0x02]))  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == bytes([0x00, 0x01]):
                y = i2c.writeto(0x61, bytes([0x03, 0x00]))  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                #crc8(x[:2]) == x[2]  etc for every 3 bytes
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid)
                if client.isconnected():
                    await client.publish(topicscd30, "%f %f %f"%(co2, temp, humid))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD30task(client))

        
async def SCD41task(client):
    try:
        i2c.writeto(0x62, bytes([0x3F, 0x86]))   # stopPeriodicMeasurement
        await asyncio.sleep_ms(500)
        i2c.writeto(0x62, bytes([0x36, 0x82]))   # getSerialNumber
        await asyncio.sleep_ms(1)
        x = i2c.readfrom(0x62, 9)
        s0, s1, s2 = struct.unpack(">HHH", bytes([x[0],x[1], x[3],x[4], x[6],x[7]]))
        print("SCD41 serial number", hex(s0), hex(s1), hex(s2))
        i2c.writeto(0x62, bytes([0x21, 0xB1]))   # startPeriodicMeasurement
        await asyncio.sleep_ms(1)
        for n in itertools_count():
            i2c.writeto(0x62, bytes([0xE4, 0xB8]))   # check ready
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x62, 3)
            if not (d[0] == 0x80 and d[1] == 0x00):
                i2c.writeto(0x62, bytes([0xEC, 0x05]))   # readMeasurement
                await asyncio.sleep_ms(1)
                x = i2c.readfrom(0x62, 9)
                co2, utemp, uhumid = struct.unpack(">HHH", bytes([x[0],x[1], x[3],x[4], x[6],x[7]]))
                temp = utemp*175.0/65536.0 - 45.0
                humid = uhumid*100.0/65536.0
                print(n, topicscd41, co2, temp, humid)
                if client.isconnected():
                    await client.publish(topicscd41, "%f %f %f"%(co2, temp, humid))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD41taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD41task(client))
        
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(SCD30task(client))
aloop.create_task(SCD41task(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


topicstem esp8266/241964
mode : sta(bc:dd:c2:24:19:64) + softAP(be:dd:c2:24:19:64)
#6 ets_task(4020f560, 28, 3fff9430, 10)
add if0
sleep disable
await connecting to: HUAWEI-B535-2F99 mosquitto.doesliverpool.xyz
0 esp8266/241964/scd30 1798.1 20.8957 45.9198
SCD41 serial number 0xf9dd 0xff07 0x3b5e
scandone
state: 0 -> 2 (b0)
state: 2 -> 3 (0)
state: 3 -> 5 (10)
add 0
aid 1
cnt 

connected with JulianS9, channel 11
dhcp client start...
4 esp8266/241964/scd30 1796.75 20.9384 45.8618
.ip:192.168.43.67,mask:255.255.255.0,gw:192.168.43.1
Checking WiFi integrity.
8 esp8266/241964/scd30 1796.55 20.9091 45.8694
9 esp8266/241964/scd41 1019 22.7559 40.3625
13 esp8266/241964/scd30 1796.76 20.9224 45.8115
.17 esp8266/241964/scd30 1798.02 20.9224 45.8969
Got reliable connection
Connecting to broker.
Connected to broker.
*** connected:
ipnumber 192.168.43.67
18 esp8266/241964/scd41 1008 22.3393 40.9775
19 esp8266/241964/scd30 1799.58 20.9224 45.871
22 esp8266/241964/scd30 1798.74 20.9224 45.8603
24 e

Traceback (most recent call last):
  File "<stdin>", line 119, in <module>
  File "uasyncio/core.py", line 1, in run_forever
  File "uasyncio/core.py", line 1, in run_until_complete
  File "uasyncio/core.py", line 1, in wait_io_event
KeyboardInterrupt: 
